In [1]:
import sys
import os
import random
import time
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QWidget, QVBoxLayout, QHBoxLayout,
    QLabel, QLineEdit, QPushButton, QTextEdit, QFileDialog
)
from Bio import SeqIO, Entrez
from Bio.Seq import Seq
from Bio.SeqUtils import MeltingTemp
from Bio.SeqRecord import SeqRecord
from Bio.Blast import NCBIWWW
from collections import Counter
import primer3

# Set up Entrez email
Entrez.email = "ghorbani.abozar@gmail.com"


class AutoPVPrimerApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("AutoPVPrimer Dashboard")
        self.resize(600, 400)

        # Main container
        self.container = QWidget()
        self.setCentralWidget(self.container)

        # Layouts
        self.layout = QVBoxLayout()
        self.container.setLayout(self.layout)

        # Input fields
        self.virus_name_label = QLabel("Virus Name:")
        self.virus_name_input = QLineEdit("Tomato Mosaic Virus")

        self.output_dir_label = QLabel("Output Directory:")
        self.output_dir_input = QLineEdit()
        self.output_dir_browse = QPushButton("Browse...")
        self.output_dir_browse.clicked.connect(self.browse_output_directory)

        # Buttons
        self.download_btn = QPushButton("Download Sequences")
        self.align_btn = QPushButton("Create Alignment & Contigs")
        self.design_btn = QPushButton("Design Primers")
        self.blast_btn = QPushButton("Run Primer BLAST")

        # Connect buttons to functions
        self.download_btn.clicked.connect(self.download_sequences)
        self.align_btn.clicked.connect(self.create_alignment_and_contigs)
        self.design_btn.clicked.connect(self.design_primers)
        self.blast_btn.clicked.connect(self.run_primer_blast)

        # Output display
        self.output_box = QTextEdit()
        self.output_box.setReadOnly(True)

        # Add widgets to layout
        self.layout.addWidget(QLabel("<h2>AutoPVPrimer Dashboard</h2>"))
        self.layout.addWidget(self.virus_name_label)
        self.layout.addWidget(self.virus_name_input)
        self.layout.addWidget(self.output_dir_label)

        output_dir_layout = QHBoxLayout()
        output_dir_layout.addWidget(self.output_dir_input)
        output_dir_layout.addWidget(self.output_dir_browse)
        self.layout.addLayout(output_dir_layout)

        button_layout = QHBoxLayout()
        button_layout.addWidget(self.download_btn)
        button_layout.addWidget(self.align_btn)
        button_layout.addWidget(self.design_btn)
        button_layout.addWidget(self.blast_btn)
        self.layout.addLayout(button_layout)

        self.layout.addWidget(QLabel("Output:"))
        self.layout.addWidget(self.output_box)

    def browse_output_directory(self):
        dir_path = QFileDialog.getExistingDirectory(self, "Select Output Directory")
        if dir_path:
            self.output_dir_input.setText(dir_path)

    def log_output(self, message):
        self.output_box.append(message)

    def download_sequences(self):
        virus_name = self.virus_name_input.text()
        output_dir = self.output_dir_input.text()

        try:
            self.log_output(f"Downloading sequences for {virus_name}...")
            search_term = f"{virus_name}[Organism] AND genome[Title]"
            handle = Entrez.esearch(db="nucleotide", term=search_term, retmax=10)
            record = Entrez.read(handle)
            handle.close()

            if record["Count"] == "0":
                self.log_output(f"No records found for {virus_name}.")
                return

            os.makedirs(output_dir, exist_ok=True)

            for seq_id in record["IdList"]:
                handle = Entrez.efetch(db="nucleotide", id=seq_id, rettype="fasta", retmode="text")
                seq_record = SeqIO.read(handle, "fasta")
                handle.close()

                filename = os.path.join(output_dir, f"{virus_name}_{seq_id}.fasta")
                SeqIO.write(seq_record, filename, "fasta")
                self.log_output(f"Sequence saved: {filename}")
        except Exception as e:
            self.log_output(f"Error downloading sequences: {e}")

    def create_alignment_and_contigs(self):
        input_folder = self.output_dir_input.text()
        output_folder = self.output_dir_input.text()

        try:
            self.log_output("Creating consensus sequence...")
            input_files = [os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.endswith(".fasta")]
            sequences = [SeqIO.read(f, "fasta") for f in input_files]

            max_length = max(len(seq) for seq in sequences)
            aligned_sequences = [str(seq.seq).ljust(max_length, '-') for seq in sequences]
            consensus_seq = ''.join(Counter(col).most_common(1)[0][0] for col in zip(*aligned_sequences))

            consensus_record = SeqRecord(Seq(consensus_seq), id="Consensus_Sequence", description="")
            output_file = os.path.join(output_folder, "Consensus_Sequence.fasta")
            SeqIO.write(consensus_record, output_file, "fasta")
            self.log_output(f"Consensus sequence saved to {output_file}")
        except Exception as e:
            self.log_output(f"Error creating consensus sequence: {e}")

    def design_primers(self):
        fasta_path = os.path.join(self.output_dir_input.text(), "Consensus_Sequence.fasta")
        output_path = self.output_dir_input.text()

        try:
            self.log_output("Designing primers...")
            record = SeqIO.read(fasta_path, 'fasta')
            primer_pairs = []

            for i in range(3):
                product_size = random.randint(250, 1000)
                target_start = random.randint(0, len(record.seq) - product_size)
                target_end = target_start + product_size
                target_sequence = record.seq[target_start:target_end]

                primers = primer3.bindings.design_primers(
                    {
                        'SEQUENCE_TEMPLATE': str(target_sequence),
                        'SEQUENCE_INCLUDED_REGION': [0, len(target_sequence)],
                    },
                    {
                        'PRIMER_OPT_SIZE': 20,
                        'PRIMER_MIN_SIZE': 18,
                        'PRIMER_MAX_SIZE': 25,
                        'PRIMER_MIN_TM': 58,
                        'PRIMER_MAX_TM': 65,
                        'PRIMER_PAIR_MAX_DIFF_TM': 2
                    }
                )

                primer_pairs.append((primers['PRIMER_LEFT_0_SEQUENCE'], primers['PRIMER_RIGHT_0_SEQUENCE']))

            output_file = os.path.join(output_path, 'primer_pairs.txt')
            with open(output_file, 'w') as file:
                file.write("Forward Primer\tReverse Primer\n")
                for fp, rp in primer_pairs:
                    file.write(f"{fp}\t{rp}\n")
            self.log_output(f"Primers saved to {output_file}")
        except Exception as e:
            self.log_output(f"Error designing primers: {e}")

    def run_primer_blast(self):
        primer_pairs_file = os.path.join(self.output_dir_input.text(), "primer_pairs.txt")
        output_dir = self.output_dir_input.text()

        try:
            with open(primer_pairs_file, 'r') as file:
                primer_pairs = [line.strip().split('\t') for line in file.readlines()[1:]]

            os.makedirs(output_dir, exist_ok=True)

            for i, (fp, rp) in enumerate(primer_pairs):
                sequence = Seq(f'{fp}{rp}')
                seq_record = SeqRecord(sequence, id=f"Primer_Pair_{i}", description="")
                result_handle = NCBIWWW.qblast("blastn", "nr", seq_record.format("fasta"))
                with open(os.path.join(output_dir, f"primer_blast_result_{i}.xml"), "w") as out_file:
                    out_file.write(result_handle.read())
                result_handle.close()
                self.log_output(f"BLAST results saved for Primer Pair {i}")
                time.sleep(2)
        except Exception as e:
            self.log_output(f"Error running Primer-BLAST: {e}")


if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = AutoPVPrimerApp()
    window.show()
    sys.exit(app.exec_())

SystemExit: 0

D:\Anaconda\Lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
